In [ ]:
# import packages
import numpy as np
from numpy import random
from scipy import stats
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split


In [ ]:
data = pd.read_csv('/Users/arberimbibaj/dataset_example_indicatorCATE.csv', header=None, index_col=[0])
data = data.to_numpy()
data

In [ ]:
N = len(data)
d = len(data[0,:]) - 3
print(N)
print(d)

In [ ]:
# train test split
random.shuffle(data)
training, test = data[:700,:], data[700:,:]

In [ ]:
# slice dataset by treatment status
training_control = training[training[:,26]==0]
training_treatment = training[training[:,26]==1]

# Y_train by treatment status
Y_train_control = training_control[:,0]
Y_train_treatment = training_treatment[:,0]

# X_train by treatment status
X_train_control = training_control[:,1:26]
X_train_treatment = training_treatment[:,1:26]

# X and Y test
X_test = test[:,1:26]
Y_test = test[:,0]

# X_train and Y_train (no split by treatment status)
X_train = training[:,1:26]
Y_train = training[:,0]

# W_train and W_test
W_train = training[:,26]
W_test = test[:,26]

# tau_test
tau_test = test[:,27]

In [ ]:
X_train_control

In [ ]:
X_train_treatment

In [ ]:
Y_train_control

In [ ]:
Y_train_treatment

In [ ]:
X_train

In [ ]:
W_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
W_test

In [ ]:
Y_test

In [ ]:
tau_test

# T-Learner

In [ ]:
# T-Learner (example with Random Forest)

# mu_0
t_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu0.fit(X_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_test)

# mu_1
t_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu1.fit(X_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_test)
# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
t_tau_hat

In [ ]:
# mean squared error
((t_tau_hat - tau_test)**2).mean()

# S-Learner

In [ ]:
### S-Learner

In [ ]:
# set training and test features for the S-Learner (it views W as no different from other X's)
X_W_train = training[:,1:27]
X_test_0 = np.concatenate((test[:,1:26],np.zeros((300,1))), axis=1)
X_test_1 = np.concatenate((test[:,1:26],np.ones((300,1))), axis=1)

In [ ]:
X_test_0

In [ ]:
X_test_1

In [ ]:
Y_train

In [ ]:
# S-learner (example with Random Forest)

# mu_x
s_learner = RandomForestRegressor(max_depth=100, random_state=0)
s_learner.fit(X_W_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
s_tau_hat


In [ ]:
# mean squared error
((s_tau_hat - tau_test)**2).mean()

# X-Learner

In [ ]:
### X-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_0_hat.fit(X_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_1_hat.fit(X_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = RandomForestClassifier(max_depth=100, random_state=0)
g_x_hat.fit(X_train,W_train)
probabilities = g_x_hat.predict_proba(X_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_test) + probas_0 * x_tau_1_hat.predict(X_test)
x_tau_hat


In [ ]:
# mean squared error (much lower here!)
((x_tau_hat - tau_test)**2).mean()

# R-Learner

In [ ]:
### R-Learner


# DR-Learner

In [ ]:
### DR-Learner

# RA-Learner

In [ ]:
### RA-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
ra_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
ra_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
ra_pseudo_outcome = W_train*(Y_train - ra_learner_mu0.predict(X_train)) + (1 - W_train)*(ra_learner_mu1.predict(X_train) - Y_train)

# tau_hat
ra_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
ra_tau_hat_learner.fit(X_train, ra_pseudo_outcome)
ra_tau_hat = ra_tau_hat_learner.predict(X_test)
ra_tau_hat

In [ ]:
# mean squared error
((ra_tau_hat - tau_test)**2).mean()

# PW-Learner

In [ ]:
### PW-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
pw_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
pw_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
pw_pseudo_outcome = (W_train/pw_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(pw_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
pw_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
pw_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
pw_tau_hat = pw_tau_hat_learner.predict(X_test)
pw_tau_hat

In [ ]:
# mean squared error
((pw_tau_hat - tau_test)**2).mean()

# F-Learner: AKA THE SAME AS PW-LEARNER!!!

In [ ]:
### F-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
f_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
f_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
f_pseudo_outcome = (W_train/f_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(f_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
f_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
f_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
f_tau_hat = f_tau_hat_learner.predict(X_test)
f_tau_hat

In [ ]:
# mean squared error
print(((f_tau_hat - tau_test)**2).mean())
print("Same as for PW-Learner")

# U-Learner

In [ ]:
### U-Learner
# estimate e_x
u_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
u_learner_e_x.fit(X_train,W_train)

# estimate mu_x
u_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
u_learner_mu_x.fit(X_train,Y_train)

# compute residuals
u_learner_residuals = (Y_train - u_learner_mu_x.predict(X_train))/(W_train - u_learner_e_x.predict_proba(X_train)[:,1])

# tau_hat - regress residuals on X
u_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
u_tau_hat_learner.fit(X_train,u_learner_residuals)

u_tau_hats = u_tau_hat_learner.predict(X_test)
u_tau_hats


In [ ]:
# mean squared error
((u_tau_hats - tau_test)**2).mean()

# Just some lasso tests

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train = poly_train.fit_transform(X_train)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)
X_poly_train

In [ ]:
from sklearn.linear_model import Lasso, LassoCV
lasso_poly = LassoCV(cv=10, random_state=0, tol=1e-2)

In [ ]:
import time

t0 = time.time()
lasso_poly.fit(X_poly_train,Y_train)

t1 = time.time()

total = t1-t0
print(f"{total:.4f} seconds")

In [ ]:
# degree 3: 10 seconds to fit (100% cpu)
# degree 4: 90 seconds to fit (100% cpu)

In [ ]:
y_predictions = lasso_poly.predict(X_poly_test)

In [ ]:
((y_predictions - Y_test)**2).mean()

# just some Neural Network test

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# make model
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model = keras.Sequential([
    keras.Input(shape=(d,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, start_from_epoch=0)

In [ ]:
print("Training Model")
training = model.fit(
    X_train,
    Y_train,
    batch_size=100,
    epochs=10,
    validation_data=(X_test, Y_test),
    callbacks=[callback] # include early stopping
)

In [ ]:
predictions

In [ ]:
print("Generate predictions for test samples")
predictions = np.reshape(model.predict(X_test),(300,))
print("predictions shape:", predictions.shape)
predictions

In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=100)

In [ ]:
((predictions - Y_test)**2).mean()